In [1]:
from pkg_resources import resource_filename
import pandas as pd
import numpy as np
import qupid
import os

In [2]:
# loads in data and reads data- change to file path
#os.getcwd()
#metadata_fpath = resource_filename("qupid", "asd.tsv")
# metadata_fpath = "asd.tsv"
# metadata = pd.read_table(metadata_fpath, sep="\t", index_col=0)

In [3]:
metadata_fpath = "sample_information_from_prep_1115.tsv"
metadata_pre = pd.read_table(metadata_fpath, sep="\t", index_col=0)

In [4]:
metadata = metadata_pre.drop(metadata_pre[metadata_pre['cat'] == 'not provided'].index)
metadata.head()

,asd,bmi,cat,dog,ibd,ibs,pku,sex,diet,race,...,pm_shared_who_posteddiscussed_on_an_online_patienthealth_platform_ie_patientslikeme,covid_family_members_put_into_self_quarantine_without_symptoms_eg_due_to_possible_exposure,covid_happened_to_you_put_into_self_quarantine_without_symptoms_eg_due_to_possible_exposure,pm_shared_pcp_changes_what_change_in_behaviorallifestyle_recommendations_eg_start_probiotic,pm_lifestyle_change_how_change_in_alcohol_or_tobacco_use_ie_starting_stopping_or_changing_amount_consumed,pm_shared_pcp_changes_what_change_in_medication_ie_starting_stopping_or_changing_the_dose_of_a_prescription,pm_lifestyle_change_how_change_in_physical_environment_ie_adopting_or_getting_rid_of_a_pet_cleaning_more_or_less_often,name,keep_sample_for_thdmi,qiita_study_id
sample_name,,,,,,,,,,,,,,,,,,,,,
10317.000001002,not provided,NaN,No,No,not provided,not provided,I do not have this condition,female,NaN,not provided,...,NaN,false,false,NaN,NaN,NaN,NaN,NaN,NaN,10317
10317.000001004,not provided,NaN,No,No,I do not have this condition,not provided,I do not have this condition,female,NaN,Caucasian,...,NaN,false,false,NaN,NaN,NaN,NaN,NaN,NaN,10317
10317.000001008,not provided,NaN,No,Yes,I do not have this condition,not provided,I do not have this condition,female,NaN,Caucasian,...,NaN,false,false,NaN,NaN,NaN,NaN,NaN,NaN,10317
10317.000001018,not provided,NaN,No,No,I do not have this condition,not provided,I do not have this condition,male,NaN,Caucasian,...,NaN,false,false,NaN,NaN,NaN,NaN,NaN,NaN,10317
10317.000001028,not provided,NaN,No,Yes,I do not have this condition,not provided,I do not have this condition,female,not applicable,Caucasian,...,NaN,false,false,NaN,NaN,NaN,NaN,NaN,NaN,10317


In [5]:
# asd_str = "Diagnosed by a medical professional (doctor, physician assistant)"
# no_asd_str = "I do not have this condition"

# background = metadata.query("asd == @no_asd_str")
# focus = metadata.query("asd == @asd_str")


In [6]:
asd_str = "Yes"
no_asd_str = "No"

background = metadata.query("cat == @no_asd_str")
focus = metadata.query("cat == @asd_str")


In [7]:

matches = qupid.shuffle(
    focus=focus,
    background=background,
    # categories=["sex", "age_years"],
    # tolerance_map={"age_years": 10},
    categories=["sex"],
    iterations=100
)

/home/l1joseph/miniforge3/envs/qiime2-amplicon-2024.2/lib/python3.8/site-packages/qupid/qupid.py:49: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for f_idx, f_val in focus.iteritems():


In [20]:
from qupid import match_by_multiple
cm = match_by_multiple(
    focus=focus,
    background=background,
    categories=["sex"]
)

/home/l1joseph/miniforge3/envs/qiime2-amplicon-2024.2/lib/python3.8/site-packages/qupid/qupid.py:49: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for f_idx, f_val in focus.iteritems():


In [21]:
from typing import List
from qupid import CaseMatchOneToMany

def random_match(
    focus: pd.DataFrame,
    background: pd.DataFrame,
    categories: List[str]
) -> CaseMatchOneToMany:
    """Randomly match samples from focus to background.

    :param focus: Samples to be matched
    :type focus: pd.DataFrame

    :param background: Metadata to match against
    :type background: pd.DataFrame

    :param categories: Categories to include as matching criteria
    :type categories: List[str]

    :returns: Randomly matched control samples
    :rtype: qupid.CaseMatchOneToMany
    """
    matches = {i: set() for i in focus.index}

    for cat in categories:
        focus_idx = focus.index.tolist()
        np.random.shuffle(focus_idx)
        for idx in focus_idx:
            match_idx = np.random.choice(background.index)
            matches[idx].add(match_idx)

    metadata = pd.concat([focus, background])
    return CaseMatchOneToMany(matches, metadata)

In [11]:
from qupid import CaseMatchOneToOne
from typing import List, Dict, Union

def random_match_one_to_one(
    focus: pd.DataFrame,
    background: pd.DataFrame,
    categories: List[str]
) -> CaseMatchOneToOne:
    """Randomly match samples from focus to background in a one-to-one manner.

    :param focus: Samples to be matched
    :type focus: pd.DataFrame

    :param background: Metadata to match against
    :type background: pd.DataFrame

    :param categories: Categories to include as matching criteria
    :type categories: List[str]

    :returns: Randomly matched control samples
    :rtype: qupid.CaseMatchOneToOne
    """
    matches = {}

    for cat in categories:
        focus_idx = focus.index.tolist()
        np.random.shuffle(focus_idx)
        for idx in focus_idx:
            match_idx = np.random.choice(background.index)
            matches[idx] = {match_idx}
            background = background.drop(match_idx)

    metadata = pd.concat([focus, background])
    return CaseMatchOneToOne(matches, metadata)

# Example usage:
# matched_samples = random_match_one_to_one(focus_df, background_df, ['category1', 'category2'])

In [22]:
rm = random_match(
    focus=focus,
    background=background,
    categories=["sex"]
)

In [13]:
rm = random_match_one_to_one(
    focus=focus,
    background=background,
    categories=["sex"]
)

In [14]:
# def random_match_table(table, match_column):
#     # Separate rows with 'yes' and 'no'
#     yes_rows = table[table[match_column] == 'Diagnosed by a medical professional (doctor, physician assistant)']
#     no_rows = table[table[match_column] == 'I do not have this condition']

#     # Check if there are at least one 'yes' and one 'no'
#     if yes_rows.empty or no_rows.empty:
#         raise ValueError("Insufficient rows with 'yes' or 'no' for matching.")

#     # Shuffle the order of rows with 'yes' and 'no'
#     yes_rows = yes_rows.sample(frac=1, random_state=42)
#     no_rows = no_rows.sample(frac=1, random_state=42)

#     # Create a DataFrame to store matched pairs
#     matched_pairs = pd.DataFrame()

#     # Iterate through each 'yes' row and match with a random 'no' row
#     for _, yes_row in yes_rows.iterrows():
#         matched_no_row = no_rows.sample(n=1).iloc[0]
#         matched_pair = pd.concat([yes_row, matched_no_row])
#         matched_pairs = matched_pairs.append(matched_pair, ignore_index=True)

#     return matched_pairs

# # Example usage:
# # Assuming 'your_table.csv' is your table with a 'match_column'
# # Replace it with the actual name of your file and column
# your_table = pd.read_csv(metadata_fpath, sep="\t", index_col=0)
# matched_pairs = random_match_table(your_table, 'asd')

# # Access the matched pairs DataFrame
# # print(matched_pairs)


In [23]:
print(len(cm.cases), len(cm.controls))
print(len(rm.cases), len(rm.controls))

147 281
147 113


In [24]:
#matched pairs only for onetomany
resultsc = cm.create_matched_pairs(iterations=100)
resultsr = rm.create_matched_pairs(iterations=100)

NoMoreControlsError: Prematurely exhausted all matching controls. Remaining cases: {'10317.000001804', '10317.000001793', '10317.000004188', '10317.000004075', '10317.000001589', '10317.000001808', '10317.000002777', '10317.000001766', '10317.000003091', '10317.000001837', '10317.000002885', '10317.000001048', '10317.000001597', '10317.000002952', '10317.000002781', '10317.000001551', '10317.000004072', '10317.000004039', '10317.000002886', '10317.000001120', '10317.000002175', '10317.000002805', '10317.000003199', '10317.000001299', '10317.000002607', '10317.000001812', '10317.000002859', '10317.000004267', '10317.000002176', '10317.000001961', '10317.000003879', '10317.000002563', '10317.000004009', '10317.000001487'}

In [25]:
print(len(resultsc[0].cases), len(resultsc[0].controls))
print(len(resultsr[0].cases), len(resultsr[0].controls))

147 147


NameError: name 'resultsr' is not defined

In [ ]:
resultsc_df = resultsc.to_dataframe()
resultsc_df.head()

In [ ]:
resultsr_df = resultsr.to_dataframe()
resultsr_df.head()

In [ ]:
rng = np.random.default_rng()
asd_mean = 4
ctrl_mean = 3.75

num_cases = len(cm.cases)
num_ctrls = len(cm.controls)

asd_values = rng.normal(asd_mean, 1, size=num_cases)
ctrl_values = rng.normal(ctrl_mean, 1, size=num_ctrls)
print(asd_values[:])

asd_values = pd.Series(asd_values, index=focus.index)
ctrl_values = pd.Series(ctrl_values, index=background.index)

print(asd_values.head())

sample_values = pd.concat([asd_values, ctrl_values])

In [ ]:
from qupid.stats import bulk_univariate_test

test_results = bulk_univariate_test(
    casematches=resultsc,
    values=sample_values,
    test="t"
)

In [ ]:
print(test_results.head())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.histplot(test_results["p-value"])

plt.show()

In [ ]:

from qupid.casematch import CaseMatchOneToMany, CaseMatchOneToOne, CaseMatchCollection

cm.save("asd_matches.one_to_many.json")  # Save all possible matches
results.save("asd_matches.100.tsv")  # Save all 100 iterations
results[15].save("asd_matches.best.json")  # Save best matching

CaseMatchOneToMany.load("asd_matches.one_to_many.json")
CaseMatchCollection.load("asd_matches.100.tsv")
CaseMatchOneToOne.load("asd_matches.best.json")